## Genearete labels

In [1]:
import pandas as pd
import nutils
import common as cm
import numpy as np
import os
from tqdm import tqdm


def calculate_stats(df, current, future):
    log_returns = np.log(df["mid_price"].shift(-60) / df["mid_price"]) * 1e4
    df[f"mean_{current}-{future}"] = (
        np.log(
            (df["mid_price"].rolling(future - current).mean().shift(-future))
            / df["mid_price"]
        )
        * 1e4
    )
    df[f"var_{current}-{future}"] = (
        log_returns.rolling(future - current).var().shift(-future)
    )
    df[f"vol_{current}-{future}"] = (
        log_returns.rolling(future - current).std().shift(-future)
    )


def calculate_stats2(df, current, future):
    df[f"max_{current}-{future}"] = (
        np.log(
            (df["mid_price"].rolling(future - current).max().shift(-future))
            / df["mid_price"]
        )
        * 1e4
    )
    df[f"min_{current}-{future}"] = (
        np.log(
            df["mid_price"].rolling(future - current).min().shift(-future)
            / df["mid_price"]
        )
        * 1e4
    )
    df[f"gap_{current}-{future}"] = (
        (
            df["mid_price"].rolling(future - current).max().shift(-future)
            - df["mid_price"].rolling(future - current).min().shift(-future)
        )
        / df["mid_price"]
        * 1e4
    )


def calculate_stats3(df, cur, future):
    results = []
    for name, group in df.groupby(df.index):
        # 计算未来收益率，超过当日数据长度时使用最后一行
        future_prices = group["mid_price"].shift(-future)
        last_valid_price = group["mid_price"].iloc[-1]
        future_prices.fillna(last_valid_price, inplace=True)
        ret = np.log(future_prices / group["mid_price"]) * 1e4
        ret = pd.DataFrame(ret, index=group.index, columns=[f"ret_{future}"])
        results.append(ret)
    return pd.concat(results)


for code in tqdm(cm.SELECTED_CODES):
    datas = cm.get_snapshot(code)
    df = datas["tickData"]
    tm = datas["timestamp"][:, 0]
    df = pd.DataFrame(df, columns=cm.COLS_SNAPSHOTS)
    df.index = tm

    # 保存原始列名
    original_columns = set(df.columns)
    df["mid_price"] = (df["AskPrice1"] + df["BidPrice1"]) / 2
    cur_futs = [(0, 60), (60, 120), (60, 300), (60, 600), (60, 6000), (120, 180)]
    for cur, futs in cur_futs:
        # calculate_stats(df, cur, futs)
        # calculate_stats2(df, cur, futs)

        df[f"ret_{futs}"] = calculate_stats3(df, cur, futs)

    # 只保留新生成的列
    new_columns = set(df.columns) - original_columns
    df = df.fillna(0)

    # 指定保存路径
    save_path = f"/mnt/disk1/multiobj_dataset/{code}/"
    os.makedirs(save_path, exist_ok=True)
    for col in new_columns:
        feature_path = os.path.join(save_path, f"{col}.npy")
        np.save(feature_path, df[col].values.reshape(-1, 1))
print(new_columns)

100%|██████████| 100/100 [13:11<00:00,  7.92s/it]

{'ret_6000', 'ret_600', 'ret_180', 'ret_60', 'ret_120', 'ret_300', 'mid_price'}
